-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [40]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
from numpy.linalg import matrix_power
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import scattered_elec_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [4]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [5]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [6]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [23]:
# %%prun

# Emax = 1e20
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# Emax = 1e10
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# print('********* Thomson regime scattered photon spectrum *********')
# ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400))
# print('********* Relativistic regime scattered photon spectrum *********')
# ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(400), inf_upp_bound=True)
# print('********* Thomson regime energy loss spectrum *********')
# engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400), nonrel=True)
# print('********* Thomson regime scattered electron spectrum *********')
# scattered_elec_spec_nonrel_ref_tf = scattered_elec_spec(lowengEe_nonrel, lowengEe_nonrel, phys.TCMB(400), nonrel=True)
# print('********* Relativistic regime scattered electron spectrum *********')
# scattered_elec_spec_rel_ref_tf = scattered_elec_spec(lowengEe_rel, lowengEe_rel, phys.TCMB(400), nonrel=False)

********* Thomson regime scattered electron spectrum *********
Computing nonrelativistic scattered electron spectrum...
Computing energy loss spectrum by beta expansion...
Computing integrals 1/6...
Computing integrals 2/6...
Computing integrals 3/6...
Computing integrals 4/6...
Computing integrals 5/6...
Computing integrals 6/6...
Integrals computed!
Computation by expansion in beta complete!
Computing energy loss spectrum by analytic series...
Computing upscattering loss spectra...
Computing series 1/7...
Computing series 2/7...
Computing series 3/7...
Computing series 4/7...
Computing series 5/7...
Computing series 6/7...
Computing series 7/7...
Computing downscattering loss spectra...
Computing series 1/7...
Computing series 2/7...
Computing series 3/7...
Computing series 4/7...
Computing series 5/7...
Computing series 6/7...
Computing series 7/7...
Computing additional spectra for new spec_type...
Computing series 1/2...
Computing series 2/2...
Nonrelativistic energy loss spectrum

In [24]:
# pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "wb"))
# pickle.dump(ics_rel_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "wb"))
# pickle.dump(engloss_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "wb"))
# pickle.dump(scattered_elec_spec_nonrel_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_nonrel_ref_tf.p", "wb"))
# pickle.dump(scattered_elec_spec_rel_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_rel_ref_tf.p", "wb"))


In [25]:
ics_thomson_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "rb"))
ics_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "rb"))
engloss_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "rb"))
# scattered_elec_spec_nonrel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_nonrel_ref_tf.p", "rb"))
# scattered_elec_spec_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_rel_ref_tf.p", "rb"))


In [41]:
%%prun
start_rs = 400
rs = start_rs
while rs > 399:
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=False
    )
    rs = np.exp(np.log(rs) - 0.002)
    print(rs)

for real?
-------- Injection Energy:  3016.701062885404
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242495428339469
-------- Energy lost (Analytic):  2.1005288026488443e-06
-------- Energy lost (Computed from photons):  2.100574657613151e-06
-------- Energy lost (Computed from electrons):  2.1005746576041595e-06
3.9131875195381971324e-188
7.654320885705090555e-176
-------- Energy lost (Computed from electrons DIRECT):  -7.654320873900173605e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary photons (Computed):  0.00013513920142325313
-------- Energy in secondary photons (Analytic):  0.00013513604173021233
-------- Energy gain from photons:  2.1036884956896493e-06
-------- Deposited Energy:  -3.1138380854897965e-09
-------- Injection Energy:  3199.0671427921043
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242493348538651
----

-------- Energy lost (Computed from electrons DIRECT):  -7.6543207880394555406e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary photons (Computed):  0.00015078936452935123
-------- Energy in secondary photons (Analytic):  0.00015078581923684656
-------- Energy gain from photons:  1.775385160178775e-05
-------- Deposited Energy:  -3.0492786523323052e-09
-------- Injection Energy:  26466.846424312855
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242493026306722
-------- Energy lost (Analytic):  1.8850476929198338e-05
-------- Energy lost (Computed from photons):  1.885103680579721e-05
-------- Energy lost (Computed from electrons):  1.8851036805322963e-05
3.9131875195381971324e-188
7.654320885705090555e-176
-------- Energy lost (Computed from electrons DIRECT):  -7.6543207821353574455e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary ph

-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242493481453392
-------- Energy lost (Analytic):  0.00012299746028810924
-------- Energy lost (Computed from photons):  0.00012300157004545121
-------- Energy lost (Computed from electrons):  0.00012300157003863887
3.9131875195381971324e-188
7.654320885705090555e-176
-------- Energy lost (Computed from electrons DIRECT):  -7.654320283193750209e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary photons (Computed):  0.00025604014021624586
-------- Energy in secondary photons (Analytic):  0.0002560329732156727
-------- Energy gain from photons:  0.00012300462728868239
-------- Deposited Energy:  -3.0572500435118236e-09
-------- Injection Energy:  163277.23780787055
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.000524249349864445
-------- Energy lost (Analytic):  0.00013146529879078992
----

-------- Injection Energy:  1432503.0845162973
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242494126879169
-------- Energy lost (Analytic):  0.002388499150644658
-------- Energy lost (Computed from photons):  0.0023885963496332163
-------- Energy lost (Computed from electrons):  0.0023885963495331453
3.9131875195381971324e-188
7.654320885705090555e-176
-------- Energy lost (Computed from electrons DIRECT):  -7.654315280051898525e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary photons (Computed):  0.0025216349143365306
-------- Energy in secondary photons (Analytic):  0.0025215346635722212
-------- Energy gain from photons:  0.002388599401408967
-------- Deposited Energy:  -3.0518758219362507e-09
-------- Injection Energy:  1519100.9828601435
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242494140466837
-------- Energy lost

-------- Injection Energy:  10538826.382994408
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.0005242362429462259
-------- Energy lost (Analytic):  0.08276512324386319
-------- Energy lost (Computed from photons):  0.08294289838648443
-------- Energy lost (Computed from electrons):  0.08294289838613622
3.9131875195381971324e-188
7.654320885705090555e-176
-------- Energy lost (Computed from electrons DIRECT):  -7.654279645301218041e-176
-------- Energy of upscattered photons:  0.00013303551292756348
-------- Energy in secondary photons (Computed):  0.08294289838648443
-------- Energy in secondary photons (Analytic):  0.08289815875679075
-------- Energy gain from photons:  0.08280986287355686
-------- Deposited Energy:  0.00013303551257935753
-------- Injection Energy:  11175921.147845225
-------- No. of Scatters (Analytic):  0.000524246424297732
-------- No. of Scatters (Computed):  0.000524235943481716
-------- Energy lost (Analytic):

...

-------- Energy lost (Computed from electrons DIRECT):  -6.2402053046602917855e-26
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  42.46741962196305
-------- Energy in secondary photons (Analytic):  42.55793387773569
-------- Energy gain from photons:  42.46728764648842
-------- Deposited Energy:  0.00013197545367660268
-------- Injection Energy:  265950978.2263856
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.0005207564062119486
-------- Energy lost (Analytic):  47.847676734592696
-------- Energy lost (Computed from photons):  47.74290585538923
-------- Energy lost (Computed from electrons):  47.74290585538256
5.566001771091685203e-17
1.4802836158316284014e-08
-------- Energy lost (Computed from electrons DIRECT):  1.9871721746879375783e-25
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  47.7429058553892

-------- Energy in secondary photons (Analytic):  3264.0328556184577
-------- Energy gain from photons:  3203.3556350305507
-------- Deposited Energy:  0.00013197527096053818
-------- Injection Energy:  2333305007.815731
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.0005180074604362334
-------- Energy lost (Analytic):  3670.4994880306654
-------- Energy lost (Computed from photons):  3598.2082871862244
-------- Energy lost (Computed from electrons):  3598.208287186222
3.1681648661831372128e-05
73922.44666659114127
-------- Energy lost (Computed from electrons DIRECT):  0.50281191855358287057
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  3598.2082871862244
-------- Energy in secondary photons (Analytic):  3670.499620006139
-------- Energy gain from photons:  3598.2081552107497
-------- Deposited Energy:  0.00013197547241361463
-------- Injection Energy:  2474358323.55

-------- Energy lost (Computed from photons):  193366.42370531883
-------- Energy lost (Computed from electrons):  193366.42370531894
0.00016143723594662820483
2815863.8428493497593
-------- Energy lost (Computed from electrons DIRECT):  122888.50745656909544
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  193366.42370531883
-------- Energy in secondary photons (Analytic):  223323.74124431922
-------- Energy gain from photons:  193366.42357334335
-------- Deposited Energy:  0.00013197559746913612
-------- Injection Energy:  19304135407.748817
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.0004974685798133871
-------- Energy lost (Analytic):  251139.86852549703
-------- Energy lost (Computed from photons):  215738.40979040123
-------- Energy lost (Computed from electrons):  215738.40979039855
0.00026856367764977509975
5040209.9593865501215
-------- Energy lost (Computed 

-------- Energy in secondary photons (Analytic):  19330145.969241727
-------- Energy gain from photons:  8148406.012369404
-------- Deposited Energy:  0.00013197772204875946
-------- Injection Energy:  179602077208.60257
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.0003876202432832256
-------- Energy lost (Analytic):  21737877.751577463
-------- Energy lost (Computed from photons):  8858826.64644026
-------- Energy lost (Computed from electrons):  8858826.646440268
0.0003407330251830241153
52488353.218066780057
-------- Energy lost (Computed from electrons DIRECT):  8708005.878375440785
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  8858826.64644026
-------- Energy in secondary photons (Analytic):  21737877.75170944
-------- Energy gain from photons:  8858826.646308284
-------- Deposited Energy:  0.00013198330998420715
-------- Injection Energy:  190459409798.3653
--

-------- Injection Energy:  1174967574445.4204
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.00022069382477931645
-------- Energy lost (Analytic):  930345156.9003863
-------- Energy lost (Computed from photons):  82482550.69564521
-------- Energy lost (Computed from electrons):  82482550.69564521
0.00021063666168341375398
165198456.79228341415
-------- Energy lost (Computed from electrons DIRECT):  82292790.67515786865
-------- Energy of upscattered photons:  0.0001319754746308612
-------- Energy in secondary photons (Computed):  82482550.69564521
-------- Energy in secondary photons (Analytic):  930345156.9005181
-------- Energy gain from photons:  82482550.69551323
-------- Deposited Energy:  0.0001319795846939087
-------- Injection Energy:  1245996896245.0999
-------- No. of Scatters (Analytic):  0.0005211103633429865
-------- No. of Scatters (Computed):  0.0002156099827490248
-------- Energy lost (Analytic):  1046227756.0555168


## Evolution

In [ ]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**7*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    eleceng)

elec_spec_init.switch_spec_type()

In [ ]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

In [ ]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 4
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz*coarsen_factor/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dlnz * coarsen_factor /phys.hubble(rs) / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    
    ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
    if positronium_phot_spec.spec_type != 'N':
        positronium_phot_spec.switch_spec_type()
        
    print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
    print('ICS secondary photon spectrum energy: ', ics_phot_spec.toteng())
    print('Upscattered photon energy: ', np.dot(in_spec_elec.N, continuum_loss)*norm_fac(rs))
    print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
    print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
    init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # For the tqdm bar
    pbar = tqdm(total = (np.log(rs) - np.log(end_rs))/dlnz/coarsen_factor)
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # Update tqdm bar.
        pbar.update(1)
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
#             print('here at least?')
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
#                 print("\n SHOULD BE ONE: ", (next_lowengelec_spec.toteng() + next_lowengphot_spec.toteng())/(
#                 np.sum(f_raw) * rate_func_eng(rs) * dt / phys.nB / rs**3), '\n')   
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
        if coarsen_factor > 1:
            prop_tf = np.zeros_like(highengphot_tf._grid_vals)
            for i in np.arange(coarsen_factor):
                prop_tf += matrix_power(highengphot_tf._grid_vals, i)
            lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
            lowengelec_tf._grid_vals = np.matmul(prop_tf, lowengelec_tf._grid_vals)
            highengphot_tf._grid_vals = matrix_power(
                highengphot_tf._grid_vals, coarsen_factor
            )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        
#         print('\n SHOULD ALSO BE ONE: ', 
#               out_highengphot_specs[-1].toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))/(
#         next_highengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengelec_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))), '\n')
        
#         print('ENERGIES:')
#         print(out_highengphot_specs[-1].toteng())
#         for i in np.arange(500):
#             print(i, highengphot_tf.in_eng[i], highengphot_tf[i].toteng())
    
#         print('\n SHOULD ALSO BE ONE (JUST HIGH ENERGY): ', 
#               out_highengphot_specs[-1].toteng()/(
#         next_highengphot_spec.toteng()), '\n')
        
#         output_eng = next_highengphot_spec.N*next_highengphot_spec.eng/norm_fac(rs)
#         output_N   = next_highengphot_spec.dNdE/norm_fac(rs)
#         utils.compare_arr([next_highengphot_spec.eng, output_eng])
#         print(output_eng[385:391]/1e9)
#         print(output_N[385:391])
#         print(next_highengphot_spec.eng[385:391])
        
#         utils.compare_arr([out_highengphot_specs[-1].N, next_highengphot_spec.N])
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002*coarsen_factor)
        
        dt = dlnz*coarsen_factor/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
            ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
            eleceng, photeng, rs, fast=True
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()
    

        next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [ ]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=10, std_soln=True
    )

In [ ]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])